In [1]:
import csv
import pandas as pd
import numpy as np
import time
print('Done')

Done


In [3]:
tweets=[]
start= time.time()
train_data= pd.read_csv('data/training.csv',names = ['Type' , 'id', 'date', 'query','user', 'tweet'], encoding='latin-1')
test_data= pd.read_csv('data/test.csv',names = ['Type' , 'id', 'date', 'query','user', 'tweet'], encoding='latin-1')
test_data = test_data.drop(['id','date','query','user'], axis=1)
train_data= train_data.drop(['id','date','query','user'], axis=1)
test_data = test_data.drop(test_data[test_data['Type']==2].index)
# print(train_data[:5])
# print(train_data[:5])
print('time',time.time()-start)
print(train_data.shape)
print(test_data.shape)

time 3.5194811820983887
(1600000, 2)
(359, 2)


In [34]:
tweets = train_data.set_index('Type')
tweets['words']= tweets['tweet'].replace(',',' ').replace('.', ' ').str.split()
tweets['word_count']= tweets['words'].str.len()
tweets = tweets.drop(['tweet'], axis=1)

# tweets_test = test_data.set_index('Type')
tweets_test=test_data
tweets_test['words']= tweets_test['tweet'].str.replace(',',' ').str.replace('.', ' ').str.split()
tweets_test['word_count']= tweets_test['words'].str.len()
tweets_test = tweets_test.drop(['tweet'], axis=1)
print(tweets[:5])
print(tweets[-5:])
print(tweets.shape)
print(tweets_test.shape)

                                                  words  word_count
Type                                                               
0     [@switchfoot, http://twitpic.com/2y1zl, -, Aww...          19
0     [is, upset, that, he, can't, update, his, Face...          21
0     [@Kenichan, I, dived, many, times, for, the, b...          18
0     [my, whole, body, feels, itchy, and, like, its...          10
0     [@nationwideclass, no,, it's, not, behaving, a...          21
                                                  words  word_count
Type                                                               
4     [Just, woke, up., Having, no, school, is, the,...          11
4     [TheWDB.com, -, Very, cool, to, hear, old, Wal...          11
4     [Are, you, ready, for, your, MoJo, Makeover?, ...          11
4     [Happy, 38th, Birthday, to, my, boo, of, alll,...          12
4     [happy, #charitytuesday, @theNSPCC, @SparksCha...           5
(1600000, 2)
(359, 3)


In [ ]:
vocab = {}
for l_words in tweets['words']:
    for word in l_words:
        vocab[word] =1
print(len(vocab))
print('Done')

total_words_vocab = len(vocab) 
print(tweets_test.shape)

In [ ]:
tweets_neg=tweets.loc[[0]]
tweets_pos=tweets.loc[[4]]
# print(tweets_neg[:10]) 
# print(tweets_pos[0:10])
# total_words_neg = tweets_neg['word_count'].sum()
# total_words_pos = tweets_pos['word_count'].sum()
# print(total_words_neg)
# print(total_words_pos)
print(tweets_neg.shape)
print('Done')

In [ ]:
dict_neg_words={}
dict_neg_words=vocab.copy()
print(vocab)
print(dict_neg_words)
# print(len(dict_neg_words))
for l_words in tweets_neg['words']:
    word_c =0
    for word in l_words:
        dict_neg_words[word] +=1
count =0
for key in dict_neg_words:
    count += dict_neg_words[key]
print(count)       
print('Done')

In [ ]:
print(12040935-1078212)
11423286-1078212

In [ ]:
dict_pos_words={}
dict_pos_words=vocab.copy()
for l_words in tweets_pos['words']:
    for word in l_words:
        dict_pos_words[word] +=1
print(len(dict_pos_words))
count =0
for key in dict_pos_words:
    count += dict_pos_words[key]
print(count)
print('Done')

In [ ]:
phi_neg = (tweets_neg.shape[0]+1)/(tweets.shape[0] +2)
phi_pos =(tweets_pos.shape[0]+1)/(tweets.shape[0]+2)
print(phi_neg)
print(phi_pos)

In [ ]:
def check_accuracy(tweets_neg, tweets_pos, tweets):
    count=0
    correct =0
    total_words_neg = tweets_neg['word_count'].sum()
    total_words_pos = tweets_pos['word_count'].sum()
    print(total_words_neg)
    print(total_words_pos)
    for tweet in tweets_neg['words']:
        theta_j_neg =0
        theta_j_pos =0
        for word in tweet:
            if word in dict_neg_words:
                theta_j_neg += np.log(dict_neg_words[word])
                theta_j_pos += np.log(dict_pos_words[word])        
        theta_j_neg -= (len(tweet) *np.log(total_words_neg))    
        theta_j_pos -= (len(tweet) *np.log(total_words_pos))
        prob_y_neg = theta_j_neg + np.log(phi_neg)
        prob_y_pos = theta_j_pos + np.log(phi_pos)
        if(prob_y_neg > prob_y_pos):
            correct += 1
    print(correct) 
    correct_neg = correct
    for tweet in tweets_pos['words']:
        theta_j_neg =0
        theta_j_pos =0
        for word in tweet:
            if word in dict_neg_words:
                theta_j_neg += np.log(dict_neg_words[word])
                theta_j_pos += np.log(dict_pos_words[word])
        theta_j_neg -= (len(tweet) *np.log(total_words_neg))    
        theta_j_pos -= (len(tweet) *np.log(total_words_pos))
        prob_y_neg = theta_j_neg + np.log(phi_neg)
        prob_y_pos = theta_j_pos + np.log(phi_pos)
        if(prob_y_neg < prob_y_pos):
            correct += 1   
    correct_pos = correct- correct_neg        
    print(correct)
    print(correct/tweets.shape[0])
    return correct_neg, correct_pos
    
    
    
start = time.time()
check_accuracy(tweets_neg, tweets_pos, tweets)
print('time', time.time()- start)
print('Done')        

In [ ]:
tweets_neg_test=tweets_test.loc[tweets_test['Type']==0]
tweets_pos_test=tweets_test.loc[tweets_test['Type']==4]
total_neg_test = tweets_neg_test.shape[0]
total_pos_test = tweets_pos_test.shape[0]
print(total_neg_test)
print(total_pos_test)
start = time.time()
correct_neg, correct_pos = check_accuracy(tweets_neg_test, tweets_pos_test, tweets_test)
print(time.time()-start)

In [ ]:
#confusion matrix
wrong_neg = total_neg_test-correct_neg
wrong_pos = total_pos_test-correct_pos
con_mat = pd.DataFrame([[correct_neg, wrong_pos],[wrong_neg, correct_pos]], columns=['neg_actual', 'pos_actual'], index=['neg_pred', 'pos_pred'])

In [ ]:
print(con_mat)

1.(d) stop word removal

In [24]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [25]:
stop_words = set(stopwords.words('english'))

In [42]:
t['tweet'].str


In [43]:
t = train_data
print(t.shape[0])
t['words']= word_tokenize(str(t['tweet']))
# tweets['word_count']= tweets['words'].str.len()
# tweets = tweets.drop(['tweet'], axis=1)
print(t[:12])

1600000


ValueError: Length of values does not match length of index

In [14]:
a=word_tokenize('i have no read the novel on which "the kite runner" is based')
print(type(a))

<class 'list'>
